# Recomendadores Híbridos

In [1]:
import imdb
import pandas as pd

# Crear instancia del objeto IMDb
ia = imdb.IMDb()

# Función para obtener información de una película por su ID
def get_movie_info(movie_id):
    movie = ia.get_movie(movie_id)
    title = movie.get('title')
    genre = movie.get('genres')
    director = movie.get('director')[0]['name'] if movie.get('director') else None
    actors = ", ".join(actor['name'] for actor in movie.get('cast')[:3]) if movie.get('cast') else None
    return title, genre, director, actors

# Obtener información de 1000 películas
movies_info = []
for i in range(1, 101):  # Itera sobre los IDs de las películas (del 1 al 1000)
    title, genre, director, actors = get_movie_info(i)
    movies_info.append({'Título': title, 'Género': genre, 'Director': director, 'Actores Principales': actors})

# Crear DataFrame de pandas con la información de las películas
df = pd.DataFrame(movies_info)

# Imprimir DataFrame
df.head()

,Título,Género,Director,Actores Principales
0,Carmencita,"[Documentary, Short]",William K.L. Dickson,Carmencita
1,Le clown et ses chiens,"[Animation, Short]",Émile Reynaud,None
2,Poor Pierrot,"[Animation, Comedy, Short, Romance]",Émile Reynaud,None
3,Un bon bock,"[Animation, Short]",Émile Reynaud,None
4,Blacksmith Scene,"[Short, Comedy]",William K.L. Dickson,"Charles Kayser, John Ott"


In [3]:
df.rename(columns={
    'Título':'titulo',
    'Género':'genero',
    'Director':'director',
    'Actores Principales':'actor'

}, inplace=True)

df.head()

,titulo,genero,director,actor
0,Carmencita,"[Documentary, Short]",William K.L. Dickson,Carmencita
1,Le clown et ses chiens,"[Animation, Short]",Émile Reynaud,None
2,Poor Pierrot,"[Animation, Comedy, Short, Romance]",Émile Reynaud,None
3,Un bon bock,"[Animation, Short]",Émile Reynaud,None
4,Blacksmith Scene,"[Short, Comedy]",William K.L. Dickson,"Charles Kayser, John Ott"


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors

# Datos de ejemplo
# Considera que estos datos podrían ser películas, libros, productos, etc.
#data = {
#    'titulo': ['El señor de los anillos', 'Harry Potter y la piedra filosofal', 'La comunidad del anillo', 'Harry Potter y la cámara secreta', 'El hobbit'],
#    'genero': ['Fantasía', 'Fantasía', 'Fantasía', 'Fantasía', 'Fantasía'],
#    'director': ['Peter Jackson', 'Chris Columbus', 'Peter Jackson', 'Chris Columbus', 'Peter Jackson'],
#    'actor': ['Elijah Wood, Ian McKellen', 'Daniel Radcliffe, Rupert Grint', 'Elijah Wood, Ian McKellen', 'Daniel Radcliffe, Rupert Grint', 'Martin Freeman']
#}

#df = pd.DataFrame(data)

# Crear matriz TF-IDF para el contenido basado en texto (título, género, director, actor)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['titulo'] + ' ' + df['genero'] + ' ' + df['director'] + ' ' + df['actor'])

# Calcular similitud de coseno entre los vectores TF-IDF
cosine_sim_content = linear_kernel(tfidf_matrix, tfidf_matrix)

# Crear matriz de vecinos más cercanos para filtrado colaborativo basado en usuario
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(tfidf_matrix)

def hybrid_recommender(title, num_recommendations=2):
    # Obtener índice del título dado
    idx = df[df['titulo'] == title].index[0]
    
    # Obtener similitudes basadas en contenido
    content_scores = list(enumerate(cosine_sim_content[idx]))
    content_scores = sorted(content_scores, key=lambda x: x[1], reverse=True)
    content_scores = content_scores[1:num_recommendations+1]  # Excluye el propio elemento
    
    # Recomendar películas basadas en contenido
    content_recommendations = [df.iloc[score[0]]['titulo'] for score in content_scores]
    
    # Recomendar películas basadas en filtrado colaborativo
    _, indices = nn_model.kneighbors(tfidf_matrix[idx], n_neighbors=num_recommendations+1)
    collab_recommendations = [df.iloc[ind]['titulo'] for ind in indices.squeeze()][1:]
    
    return content_recommendations, collab_recommendations

# Ejemplo de uso
title = 'El hobbit'
content_recs, collab_recs = hybrid_recommender(title)
print("Recomendaciones basadas en contenido:")
print(content_recs)
print("\nRecomendaciones basadas en filtrado colaborativo:")
print(collab_recs)


TypeError: can only concatenate str (not "list") to str